In [158]:
import json
import pandas as pd
import sys
import random
sys.path.append('./DnD-Battler/')
import DnD

In [69]:
url = "https://raw.githubusercontent.com/oganm/dnddata/master/data-raw/dnd_chars_all.json"
df = pd.read_json(url)
df = df.transpose()

In [167]:
ignored = ['ip', 'finger', 'date', 'hash', 'choices', 'name', 'country']
dont_process = ['weapons', 'spells', 'skills']
clean_df = pd.json_normalize(df['name'])
clean_df = clean_df['alias'].str[0]

for key in df.keys():
    if key not in ignored:
        if key in dont_process:
            data = df[key].reset_index(drop=True)
        else:
            try: # try doing a json_normalize, which will fail if it's not a json
                data = pd.json_normalize(df[key])
                for col in data.keys(): # unpack the singletons in each json column
                    data[col] = data[col].str[0]
            except AttributeError: 
                data = df[key].str[0]
                data = data.reset_index(drop=True) # use integer index to conform with json_normalize index

        clean_df = pd.concat([clean_df, data], axis=1)

num_spells = df['spells'].apply(len).reset_index(drop=True)
clean_df['num_spells'] = num_spells

# add generic "primary class" column based on highest level class of character
class_list = ['Artificer.level','Barbarian.level','Bard.level','Cleric.level','Druid.level','Fighter.level','Monk.level','Paladin.level','Ranger.level','Rogue.level','Sorcerer.level','Warlock.level','Wizard.level']
class_df = clean_df[class_list]

primary_class = class_df.idxmax(axis=1)
class1 = list(primary_class.str.split('.').str[0])
clean_df['class1'] = class1

# add generic "primary class level" column/label
primary_class_level = class_df.max(axis=1)
clean_df['level1'] = primary_class_level

# one-hot encode castingStat
castingStat = pd.get_dummies(clean_df["castingStat"], prefix='castingStat')
clean_df = pd.concat([clean_df, castingStat], axis = 1)

# one-hot encode processedRace 
processedRace = pd.get_dummies(clean_df["processedRace"], prefix='processedRace')
clean_df = pd.concat([clean_df, processedRace], axis = 1)

# add hp / level
clean_df['hp_per_lvl'] = clean_df['HP'] / clean_df['level1']

In [193]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(len(clean_df.iloc[0]))
    
# weapons = clean_df.iloc[0]['weapons']
# for weapon in weapons:
#     print(weapons[weapon]['processedWeapon'])
    
# clean_df
# pd.json_normalize(clean_df.iloc[0]['weapons'])

165


In [190]:
char = clean_df.iloc[0]
donald = DnD.Creature("Donny", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )

donjr = DnD.Creature("djr", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )

mike = DnD.Creature("mike", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='T' )


joe = DnD.Creature("joe", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='yuh' )

kamala = DnD.Creature("kamala", abilities={'dex': char['Dex'], 'str': char['Str'], 'cha': char['Cha'], 'con': char['Con'], 
                                  'int': char['Int'], 'wis': char['Wis']}, 
             ac=char.AC, hp=char.HP, attack_parameters='flail', level=char['level'], alignment='yuh' )
print(donald.__dict__)

56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
56 2 6
choice HD... 6.285714285714286
{'log': 'Weapon matched by str to flail\n', 'settings': {}, 'name': 'Donny', 'level': 6, 'xp': None, 'proficiency': 3, 'able': 1, 'ability_bonuses': {'str': 0, 'dex': 4, 'con': 2, 'wis': 4, 'int': 1, 'cha': -2}, 'abilities': {'str': 10, 'dex': 18, 'con': 15, 'wis': 18, 'int': 13, 'cha': 6}, 'hd': <DnD.Dice object at 0x7fbca7909160>, 'hp': 56, 'starting_hp': 56, 'ac': 18, 'initiative': <DnD.Dice object at 0x7fbca79093d0>, 'sc_ab': 'con', 'starting_healing_spells': 0, 'healing_spells': 0, 'attacks': [{'name': 'flail', 'damage': <DnD.Dice object at 0x7fbca7909310>, 'attack': <DnD.Dice object at 0x7fbca7909760>}], 'hurtful': 8.5, 'attack_parameters': [['flail', 7, 4, 8]], 'alt_attack': {'name': None, 'attack': None}, 'alignment': 'T', 'tally': {'damage': 0, 'hits': 0, 'dead': 0, 'misses': 0, 'battles': 

In [192]:
arena = DnD.Encounter(donald, joe, kamala, donjr, mike)  #Encounter accepts both Creature and strings.
# print(arena.go_to_war(10000).json())
print(arena.battle(verbose =1).json())


{"prediction": "Rough a priori predictions:\n> T= expected rounds to survive: 1991.11; crudely normalised: 0%\n> yuh= expected rounds to survive: 597.33; crudely normalised: 0%\n", "battles": 1, "rounds": 13, "notes": "", "team_names": ["T", "yuh"], "team_victories": [1, 0], "team_perfects": [0, 0], "team_close": [0, 0], "combattant_names": ["joe", "Donny", "mike"], "combattant_alignments": ["yuh", "T", "T"], "combattant_damage_avg": [40.0, 0.0, 62.0], "combattant_hit_avg": [4.0, 0.0, 5.0], "combattant_miss_avg": [8.0, 0.0, 7.0], "combattant_rounds": [0.9230769230769231, 1.0, 0.9230769230769231], "sample_encounter": "==NEW BATTLE==\nTurn order:\n['joe', 'Donny', 'mike']\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging\nmike attacks joe with flail\njoe took 10 of damage. Now on 46 hp.\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging\nmike attacks joe with flail\njoe took 6 of damage. Now on 40 hp.\n**NEW ROUND**\njoe attacks Donny with flail\nDonny is dodging

In [173]:
levelGroups = ['1-3', '4-7', '8-11', '12-15', '16-18', '19-20']
# clean_df[~clean_df['levelGroup'].isin(levelGroups)]['levelGroup']
for group in levelGroups:
    grouped_chars = clean_df[clean_df['levelGroup'] == group]
    group_size = random.randint(2, 6)
    sample = grouped_chars.sample(group_size)
    print(sum(sample['level']) / group_size)
    

1.5
5.666666666666667
9.0
12.75
16.4
19.833333333333332
